## Zero-Fare

In [1]:
import altair as alt
import utilities
from calitp_data_analysis.sql import to_snakecase
from shared_utils import styleguide

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import pandas as pd
from babel.numbers import format_currency
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/consolidated_applications/"
FILE_NAME = "Con_App_Cleaned.xlsx"

In [4]:
con_app = pd.read_excel(
    f"{GCS_FILE_PATH}{FILE_NAME}", sheet_name="cleaned_unpivoted_data"
)

In [5]:
drop_cols = [
    "application_name",
    "year",
    "project_upin",
    "project_line_item__ali_",
    "is_stimulus",
    "consolidated_application",
    "district",
    "short_description",
]

In [6]:
con_app = con_app.drop(columns=drop_cols)

In [7]:
fare_strings = "free free|free fares|reduced|reduction|free"

In [8]:
free_fare = (
    con_app[con_app.project_description.str.contains(fare_strings)]
).reset_index(drop=True)

In [9]:
col_right_order = [
    "organization_name",
    "project_category",
    "application_status",
    "full_district_name",
    "project_description",
    "total_expenses",
    "fully_funded",
    "_5311_funds",
    "other_state_funds",
    "state_total",
    "local_total",
    "total_state_federal_local_funding",
    "total_state_fed_only",
    "_5311_f__funds",
    "_5311_cmaq_funds",
    "_5339_funds",
    "federal_total",
    "other_fed_funds_total",
    "lctop__state__funds",
    "sb1__state_of_good_repair__state__funds",
    "transit_development_act__state__funds",
]

In [10]:
fund_cols = [
    "total_expenses",
    "_5311_funds",
    "other_state_funds",
    "state_total",
    "local_total",
    "total_state_federal_local_funding",
    "total_state_fed_only",
    "_5311_f__funds",
    "_5311_cmaq_funds",
    "_5339_funds",
    "federal_total",
    "other_fed_funds_total",
    "lctop__state__funds",
    "sb1__state_of_good_repair__state__funds",
    "transit_development_act__state__funds",
]

In [11]:
specific_fund_cols = [
    "_5311_funds",
    "_5311_f__funds",
    "_5311_cmaq_funds",
    "_5339_funds",
    "federal_total",
    "other_fed_funds_total",
    "lctop__state__funds",
    "sb1__state_of_good_repair__state__funds",
    "transit_development_act__state__funds",
]

In [12]:
for i in specific_fund_cols:
    print(f"{i}: {free_fare[i].sum()}")

_5311_funds: 0
_5311_f__funds: 0
_5311_cmaq_funds: 0
_5339_funds: 0
federal_total: 0
other_fed_funds_total: 0
lctop__state__funds: 13779138.0
sb1__state_of_good_repair__state__funds: 0
transit_development_act__state__funds: 0.0


In [13]:
f"The total cost of projects is ${free_fare.total_expenses.sum()}" 

'The total cost of projects is $13429478'

In [14]:
f"The total award requested is ${free_fare.lctop__state__funds.sum()}" 

'The total award requested is $13779138.0'

In [15]:
free_fare.full_district_name.value_counts()

District 6: Fresno / Bakersfield          3
District 2: Redding                       2
District 1: Eureka                        1
District 10: Stockton                     1
District 12: Orange County                1
District 8: San Bernardino / Riverside    1
District 3: Marysville / Sacramento       1
District 7: Los Angeles                   1
District 4: Bay Area / Oakland            1
Name: full_district_name, dtype: int64

In [16]:
free_fare.application_status.value_counts()

Submitted        9
Not Submitted    3
Name: application_status, dtype: int64

In [17]:
free_fare.fully_funded.value_counts()

Fully funded                      9
Funding exceeds total expenses    2
Not fully funded                  1
Name: fully_funded, dtype: int64

In [18]:
free_fare2 = free_fare.copy()

In [19]:
# Format currency
for i in fund_cols:
    free_fare2[i] = free_fare2[i].apply(
        lambda x: format_currency(x, currency="USD", locale="en_US")
    )

In [20]:
free_fare2 = free_fare2[col_right_order]

In [21]:
def clean_up_columns(df):
    df.columns = df.columns.str.replace("_", " ").str.strip().str.title()
    return df

In [22]:
free_fare2 = clean_up_columns(free_fare2)

In [23]:
free_fare2.style.where(lambda val: 'Not Submitted' in str(val), 'color: red')

/tmp/ipykernel_833/3421946405.py:1: FutureWarning: this method is deprecated in favour of `Styler.applymap()`
  free_fare2.style.where(lambda val: 'Not Submitted' in str(val), 'color: red')


,Organization Name,Project Category,Application Status,Full District Name,Project Description,Total Expenses,Fully Funded,5311 Funds,Other State Funds,State Total,Local Total,Total State Federal Local Funding,Total State Fed Only,5311 F Funds,5311 Cmaq Funds,5339 Funds,Federal Total,Other Fed Funds Total,Lctop State Funds,Sb1 State Of Good Repair State Funds,Transit Development Act State Funds
0,City of Arcata,Operating,Submitted,District 1: Eureka,operating assistance - free fares,"$13,595.00",Fully funded,$0.00,$0.00,"$13,595.00",$0.00,"$13,595.00","$13,595.00",$0.00,$0.00,$0.00,$0.00,$0.00,"$13,595.00",$0.00,$0.00
1,City of Tracy,Operating,Not Submitted,District 10: Stockton,free fares program (lctop only),$0.00,Funding exceeds total expenses,$0.00,$0.00,"$67,412.00",$0.00,"$67,412.00","$67,412.00",$0.00,$0.00,$0.00,$0.00,$0.00,"$67,412.00",$0.00,$0.00
2,Kern Regional Transit,Operating,Submitted,District 6: Fresno / Bakersfield,operating assistance- lctop (2022 free fares),"$447,513.00",Fully funded,$0.00,$0.00,"$447,513.00",$0.00,"$447,513.00","$447,513.00",$0.00,$0.00,$0.00,$0.00,$0.00,"$447,513.00",$0.00,$0.00
3,Kings County Area Public Transit Agency,Operating,Submitted,District 6: Fresno / Bakersfield,operating assistance (lctop free fares or reduced fares) #1,"$64,513.00",Fully funded,$0.00,$0.00,"$64,513.00",$0.00,"$64,513.00","$64,513.00",$0.00,$0.00,$0.00,$0.00,$0.00,"$64,513.00",$0.00,$0.00
4,Kings County Area Public Transit Agency,Operating,Submitted,District 6: Fresno / Bakersfield,operating assistance (lctop free fares or reduced fares) #2,"$70,000.00",Fully funded,$0.00,$0.00,"$70,000.00",$0.00,"$70,000.00","$70,000.00",$0.00,$0.00,$0.00,$0.00,$0.00,"$70,000.00",$0.00,$0.00
5,Orange County Transportation Authority,Operating,Submitted,District 12: Orange County,youth ride free program (lctop only) #2,"$8,027,983.00",Fully funded,$0.00,$0.00,"$8,027,983.00",$0.00,"$8,027,983.00","$8,027,983.00",$0.00,$0.00,$0.00,$0.00,$0.00,"$8,027,983.00",$0.00,$0.00
6,Plumas County Transportation Commission,Operating,Submitted,District 2: Redding,lctop #2 - free fare days,"$22,825.00",Not fully funded,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
7,Redding Area Bus Aurthority,Operating,Not Submitted,District 2: Redding,free fares for students (lctop) #1,"$22,912.00",Fully funded,$0.00,$0.00,"$22,912.00",$0.00,"$22,912.00","$22,912.00",$0.00,$0.00,$0.00,$0.00,$0.00,"$22,912.00",$0.00,$0.00
8,Riverside County Transportation Commission,Operating,Submitted,District 8: San Bernardino / Riverside,riverside county free rail pass program (lctop only),"$2,406,486.00",Fully funded,$0.00,$0.00,"$2,406,486.00",$0.00,"$2,406,486.00","$2,406,486.00",$0.00,$0.00,$0.00,$0.00,$0.00,"$2,406,486.00",$0.00,$0.00
9,Sacramento Regional Transit District,Capital,Submitted,District 3: Marysville / Sacramento,free fare october,"$1,292,000.00",Fully funded,$0.00,$0.00,"$1,292,000.00",$0.00,"$1,292,000.00","$1,292,000.00",$0.00,$0.00,$0.00,$0.00,$0.00,"$1,292,000.00",$0.00,$0.00


In [24]:
# free_fare2.to_excel(f"{GCS_FILE_PATH}zero_fare.xlsx", sheet_name="zero_fare", index=False)